In [1]:
#%matplotlib inline
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cross_validation import train_test_split

Using Theano backend.


In [2]:
!which python

/home/ubuntu/anaconda2/bin/python


[Habrahabr tutorial](https://habrahabr.ru/company/ods/blog/325432/)

[Keras documentation](https://keras.io/getting-started/sequential-model-guide/)

### Оценка вина по характеристикам

загружаем данные

In [11]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
df = pd.read_csv(url, sep=';')
df.shape

(1599, 12)

In [12]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [13]:
sns.distplot(df.quality, kde=False)

Преобразуем к трехклассовой модели

In [14]:
df.quality = df.quality.where(df.quality > 5, 5)
df.quality = df.quality.where(df.quality < 7, 7)
X = df.drop(['quality'], axis=1)
y = df.quality
sns.distplot(y)

In [15]:
(X_train, X_test, y_train, y_test) = train_test_split(X.values, y.values, train_size=0.75, random_state=0)
print(X_train.shape)
print(X_test.shape)

(1199, 11)
(400, 11)


### Строим модель Keras

In [16]:
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, Dropout

#### Standard Sequential model

In [29]:
## inputs = 11
## outputs = 3
## HIDDEN_NEURONS_NUM = 50 ???

def one_hot(data, nclasses):    
    input_vector = data[:, np.newaxis] - 5 #нормализуем
    return keras.utils.to_categorical(input_vector, nclasses) #one_hot (для перекрестной энтропии?)

y_train_m = one_hot(y_train, 3)
y_test_m = one_hot(y_test, 3)

In [50]:
model = Sequential()
model.add(Dense(200, activation='sigmoid', input_dim=11))
model.add(Dense(3, activation='softmax'))

In [51]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [52]:
model.fit(X_train, y_train_m, 
          epochs=1, 
          batch_size=128)

Epoch 1/1
1199/1199 [==============================] - 0s - loss: 1.2185 - acc: 0.3428     


In [53]:
score = model.evaluate(X_test, y_test_m, batch_size=512)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

400/400 [==============================] - 0s
('Test loss:', 0.9548075795173645)
('Test accuracy:', 0.55500000715255737)


In [54]:
#from IPython.display import SVG
#from keras.utils.vis_utils import model_to_dot
#!apt install graphviz
#SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))